In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.append("../")

In [3]:
from fof.encdec import EncoderDecoderModel
from fof.dataloader import ScicapDataModule
from pathlib import Path
import wandb

In [4]:
run = wandb.init(project="figuring-out-figures")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: figuring-out-figures (use `wandb login --relogin` to force relogin)


In [5]:
import pytorch_lightning as pl

datamodule = ScicapDataModule(
    "First-Sentence",
    batch_size=32,
    tokenizer=None,
    num_workers=32,
    root=Path("../scicap_data"),
    caption_type="orig")
trainer = pl.Trainer(gpus=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Initializing SCICAP training dataset
Initializing SCICAP testing dataset
Initializing SCICAP validation dataset
Time taken: 0.05723261833190918


In [6]:
def test_model(artifact_str: str = None, ckpt_path: str = None, use_test = False):
    if artifact_str is not None:
        artifact = run.use_artifact(artifact_str, type="model")
        artifact_dir = artifact.download()
        ckpt_path = Path(artifact_dir) / "model.ckpt"
    model = EncoderDecoderModel.load_from_checkpoint(ckpt_path)
    if use_test:
        trainer.test(model, datamodule=datamodule)
    else:
        trainer.validate(model, datamodule=datamodule)

In [21]:
# Text features only, best model
test_model(ckpt_path="/data/kevin/arxiv/evaluation_checkpoints/text-features-only.ckpt")

NameError: name 'test_model' is not defined

In [ ]:
# Image features (CLIP) only, best model
test_model(artifact_str="figuring-out-figures/figuring-out-figures/model-yw5qm3wp:v8")

In [ ]:
# GPT2 + text dropout
test_model(artifact_str="figuring-out-figures/figuring-out-figures/model-1r9v2s51:v2", use_test=True)

In [8]:
# DistilGPT top p caption
test_model(artifact_str="figuring-out-figures/figuring-out-figures/model-27py12gz:v1", use_test=True)

wandb: Downloading large artifact model-27py12gz:v1, 2532.30MB. 1 files... Done. 0:0:0
Some weights of the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.11.self_attn.q_proj.weight', 'text_model.encoder.layers.2.mlp.fc1.weight', 'text_model.encoder.layers.0.self_attn.v_proj.weight', 'text_model.encoder.layers.1.self_attn.k_proj.bias', 'text_model.encoder.layers.2.mlp.fc2.weight', 'text_model.encoder.layers.10.self_attn.out_proj.bias', 'text_model.encoder.layers.3.self_attn.k_proj.bias', 'text_model.encoder.layers.0.self_attn.k_proj.bias', 'text_model.encoder.layers.11.self_attn.out_proj.weight', 'text_model.encoder.layers.3.self_attn.k_proj.weight', 'text_model.encoder.layers.2.layer_norm1.bias', 'text_model.encoder.layers.3.self_attn.q_proj.weight', 'text_model.encoder.layers.7.self_attn.out_proj.bias', 'text_model.encoder.layers.4.self_attn.v_proj.weight', 'text_model.encoder.layers.7.self_attn.q_proj.weig

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/bleu_score': 4.628124237060547,
 'test/loss': 0.8995480537414551,
 'test/perplexity': 2.5234551429748535,
 'test/rouge_L_score': 0.25070007358365387,
 'test/rouge_score': 0.27026674524935257}
--------------------------------------------------------------------------------
